In [1]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging

import matplotlib.pyplot as plt

In [2]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

In [3]:
import h5py

In [4]:
START = datetime(1980, 1, 1)
END = datetime(2020, 4, 23)
msft = yf.Ticker("MSFT")
data = msft.history(start=START, end=END)

In [6]:
data['lag_close'] = data['Close'].shift(1)
data['Indicator'] = np.where(data['Close'] > data['lag_close'], 1, 0)

In [9]:
sub_dta = data.iloc[-30:]

In [10]:
def dta_to_candlestick(data):
    l = len(data)
    # Make candlestick picture
    layout = go.Layout(xaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False,
                                  rangeslider=dict(visible=False)),
                       yaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False),
                       #width=500,
                       #height=500,
                       paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(data=[go.Candlestick(x=np.linspace(1,l,l),
                                         open=data.Open,
                                         high=data.High,
                                         low=data.Low,
                                         close=data.Close)],
                    layout=layout)
    fig.write_image("images/fig.png")

    # Convert to numpy array
    im = Image.open('images/fig.png')
    data = np.asarray(im)

    # Return the first channel of the image
    return data[:, :, 0]

In [12]:
cd_dta = dta_to_candlestick(sub_dta)

In [16]:
x = []
y = []
for i in range(30, data.shape[0]):
    sub_dta = data.iloc[i - 30:i]

    y_i = data.iloc[i]['Indicator']
    x_i = dta_to_candlestick(sub_dta)

    y.append(y_i)
    x.append(x_i)

In [19]:
dta_x = np.stack(x, axis=2)

In [20]:
dta_x.shape

(500, 700, 8568)

In [21]:
hf = h5py.File('data.h5', 'w')

In [22]:
hf.create_dataset('dataset_1', data=dta_x)

<HDF5 dataset "dataset_1": shape (500, 700, 8568), type "|u1">

In [23]:
hf.close()

In [25]:
np.save('MSFT_x.npy', dta_x)

In [26]:
np.savez_compressed('MSFT_test', a=dta_x, b=np.array(y))

In [3]:
loaded = np.load('MSFT_test.npz')

In [4]:
dta_x = loaded['a']

In [5]:
dta_x.shape

(500, 700, 8568)

In [6]:
dta_y = loaded['b']
dta_y.shape

(8568,)

## Pytorch Machine Learning Shit

In [7]:
from ResNet_CNN import *

In [44]:
import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision

In [45]:
from functools import partial
from collections import OrderedDict

In [6]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding = (self.kernel_size[0] // 2, self.kernel_size[1] // 2)
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)      

In [7]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [8]:
def activation_func(activation):
    return nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

In [9]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()
        
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels
    

In [10]:
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1, stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels)) if self.should_apply_shortcut else None
    
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels
        

In [11]:
ResNetResidualBlock(32, 64)

ResNetResidualBlock(
  (blocks): Identity()
  (activate): ReLU(inplace=True)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [12]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs),
                        nn.BatchNorm2d(out_channels))

In [13]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),)

In [14]:
dummy = torch.ones((1,32, 10, 10))

block = ResNetBasicBlock(32, 64)
block(dummy).shape

torch.Size([1, 64, 10, 10])

In [15]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),)

In [16]:
print(block)

ResNetBasicBlock(
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (activate): ReLU(inplace=True)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [17]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBottleNeckBlock, n=1, *args, **kwargs):
        super().__init__()
        downsampling = 2 if in_channels != out_channels else 1
        self.blocks = nn.Sequential(
            block(in_channels, out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion,
                   out_channels, downsampling=1, *args, **kwargs) for _ in range(n-1)])
        
    def forward(self, x):
        x = self.blocks(x)
        return x

In [18]:
dummy = torch.ones((1,64, 48,48))

layer = ResNetLayer(64, 128, block=ResNetBottleNeckBlock, n=1)
layer(dummy).shape

torch.Size([1, 512, 24, 24])

In [19]:
class ResNetEncoder(nn.Module):
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[1, 1, 1, 1], activation='relu',
                 block=ResNetBottleNeckBlock, *args, **kwargs):
        super().__init__()
        self.block_sizes = blocks_sizes

        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.block_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.block_sizes[0]),
            activation_func(activation),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))

        self.blocks = nn.ModuleList([
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, block=block, *args,
                        **kwargs),
            *[ResNetLayer(in_channels * block.expansion, out_channels, n=n, activation=activation, block=block, *args,
                          **kwargs)
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])
              ]
        ])

    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)

        return x

In [20]:
class ResNetDecoder(nn.Module):
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.ModuleList([
            nn.Linear(in_features, 500),
            nn.Dropout2d(0.5),
            nn.ReLU(),
            nn.Linear(500, 100),
            nn.Dropout2d(0.5),
            nn.ReLU(),
            nn.Linear(100, 25),
            nn.Dropout2d(0.5),
            nn.ReLU(),
            nn.Linear(25,1),
            nn.Sigmoid()
        ])
        

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        for block in self.decoder:
            x = block(x)
        return x

In [21]:
class ResNet(nn.Module):
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResNetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x

In [22]:
def res_conv1(in_channels, n_classes, block=ResNetBottleNeckBlock, *args, **kwargs):
    return ResNet(in_channels, n_classes, block=block, deepths=[1,1,1,1], *args, **kwargs)

In [23]:
import torchvision.models as models

In [10]:
from torchsummary import summary

In [8]:
model = res_conv1(1, 500)

In [9]:
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [11]:
summary(model.cuda(), (1,64,64))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 2048, 2, 2]          --
|    └─Sequential: 2-1                   [-1, 64, 16, 16]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          3,136
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─MaxPool2d: 3-4               [-1, 64, 16, 16]          --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 2048, 1, 1]          --
Total params: 9,086,815
Trainable params: 9,086,815
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.00
Params size (MB): 34.66
Estimated Total Siz

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 2048, 2, 2]          --
|    └─Sequential: 2-1                   [-1, 64, 16, 16]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          3,136
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─MaxPool2d: 3-4               [-1, 64, 16, 16]          --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 2048, 1, 1]          --
Total params: 9,086,815
Trainable params: 9,086,815
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.00
Params size (MB): 34.66
Estimated Total Siz

## Putting everything together

In [23]:
X = dta_x[:,:,568:1568]

In [34]:
Y = dta_y[568:1568]

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
import torchvision.transforms as transforms

In [20]:
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
from torch.autograd import Variable
from torch.optim import Adam, SGD

In [35]:
X = X.reshape((-1, 500, 700))
Y = Y.reshape((-1, 1))

In [36]:
X.shape

(1000, 500, 700)

In [37]:
Y.shape

(1000, 1)

In [38]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)

In [39]:
(train_X.shape, train_Y.shape), (val_X.shape, val_Y.shape)

(((900, 500, 700), (900, 1)), ((100, 500, 700), (100, 1)))

In [50]:
train_X = train_X.reshape(-1, 1, 500, 700)
train_X = torch.from_numpy(train_X)

train_Y = train_Y.astype(int)
train_Y = torch.from_numpy(train_Y)

TypeError: from_numpy() takes no keyword arguments

In [41]:
val_X =  val_X.reshape(-1, 1, 500, 700)
val_X = torch.from_numpy(val_X)

val_Y = val_Y.astype(int)
val_Y = torch.from_numpy(val_Y)

In [42]:
model = res_conv1(1, 500)

In [46]:
lr = 0.01
optimizer = Adam(model.parameters(), lr=lr)

criterion = CrossEntropyLoss()

Don't run because CUDA is out of memory

In [53]:
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [54]:
def train(epochs):
    model.train()
    tr_loss = 0
    # dataset
    x_train, y_train = Variable(train_X), Variable(train_Y)
    x_val, y_val = Variable(val_X), Variable(val_Y)

    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()
  
    optimizer.zero_grad()
    
    output_train = model(x_train)
    output_val = model(x_val)
    
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    if epochs % 2 == 0:
        print('Epoch: ', epochs+1, '\t', 'train loss: ', loss_train, '\t', 'val loss: ', loss_val)

In [55]:
n_epochs = 10
train_losses = []
val_losses = []

for epochs in range(n_epochs):
    train(epochs)

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same